In [1]:
# Подключение библиотек

import os 
import pandas as pd
from nltk.stem.snowball import SnowballStemmer
import string

In [2]:
# Путь к файлам с данными
path = 'data'

# Переход в папку с данным путём 
os.chdir(path) 

# Считывание всех файлов с данными для датасета в список
datasets = [pd.read_csv(os.path.basename(file), low_memory=False) for file in os.listdir() if file.startswith('data')]

In [3]:
# Объединение данных в один датасет

df = pd.concat(datasets,ignore_index=True).drop_duplicates().sort_values('Number').reset_index(drop=True)
df = df.drop(columns='Unnamed: 0')

In [4]:
df.head()

,Number,Barcode,Name
0,1,3165140852395,Далекомір BOSCH Zamo II (металлическая упаковк...
1,1,4044996135779,"Отвертка, SL5 х 100 мм, CrV, двухкомпонентная ..."
2,1,4606008386861,Пенал школьный 46450/25 Собачки малиновый Феникс
3,2,4811208002736,Анкер-клин 6х60 мм STARFIX SMP1-83721-1
4,2,4811208002736,Анкер-клин 6х60 мм STARFIX SMP1-83721-1


In [5]:
# Стемминг текстовых описаний

stemmer = SnowballStemmer('english')

df['Name'] = df['Name'].apply(lambda x: ''.join([stemmer.stem(y) for y in x]))

In [6]:
df.head()

,Number,Barcode,Name
0,1,3165140852395,далекомір bosch zamo ii (металлическая упаковк...
1,1,4044996135779,"отвертка, sl5 х 100 мм, crv, двухкомпонентная ..."
2,1,4606008386861,пенал школьный 46450/25 собачки малиновый феникс
3,2,4811208002736,анкер-клин 6х60 мм starfix smp1-83721-1
4,2,4811208002736,анкер-клин 6х60 мм starfix smp1-83721-1


In [7]:
# Удаление символов пунктуации

df['Name'] = df['Name'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))

In [8]:
df.head()

,Number,Barcode,Name
0,1,3165140852395,далекомір bosch zamo ii металлическая упаковка...
1,1,4044996135779,отвертка sl5 х 100 мм crv двухкомпонентная рук...
2,1,4606008386861,пенал школьный 4645025 собачки малиновый феникс
3,2,4811208002736,анкерклин 6х60 мм starfix smp1837211
4,2,4811208002736,анкерклин 6х60 мм starfix smp1837211


Формула для метрики похожести:

$\mathrm{similarity} = \frac{|\bigcap_{i=1}^{k} S_k|}{|\bigcup_{i=1}^{k} S_k|}$,

где $S_1, \dots, S_k$ -- текстовые описания (строки из слов).

In [9]:
# Функция для получения множества слов по текстовому описанию
# Вход: текстовое описание
# Выход: множество слов в текстовом описании

def words(sentence):
    sentence += ' '
    w = []
    start = 0
    end = 0
    while end <= len(sentence) - 2:
        if sentence[end+1] == ' ':
            end = end + 2
            w.append(sentence[start:end-1])
            start = end
        else:
            end = end + 1
    return set(w)

# Функция для рассчёта метрики похожести двух текстовых описаний
# Вход: список текстовых описаний
# Выход: список из множеств слов в текстовых описаниях (слово = подстрока, разделённая пробелами)

def similarity(sentences):
    # Преобразование списка описаний к списку множеств слов
    sentences_trans = [words(sentence) for sentence in sentences]
    # Число элементов в пересечении
    size_of_inter = len(set.intersection(*sentences_trans))
    # Число элементов в объединении
    size_of_union = len(set.union(*sentences_trans))
    # Метрика похожести
    similarity_metric = size_of_inter / size_of_union
    # Вывод значения похожести в процентах
    return float('{:.2f}'.format(similarity_metric*100))

In [10]:
# Новая таблица со сгруппированными текстовыми описаниями для каждого номера карточки

data = df.drop(columns='Barcode').groupby('Number').agg(set)

In [11]:
data.head()

,Name
Number,
1,{отвертка sl5 х 100 мм crv двухкомпонентная ру...
2,"{анкерклин 6х60 мм starfix smp1837211, чохол t..."
4,"{скалка деревянная фигурная сердечки 283см, см..."
5,{светильник loft lsp9805}
7,{32 жк монитор dell p3222qe 505267 }


In [12]:
# Вычисление похожести для каждого набора текстовых описаний

data['Similarity'] = data['Name'].apply(similarity)

In [13]:
data.head()

,Name,Similarity
Number,,
1,{отвертка sl5 х 100 мм crv двухкомпонентная ру...,0.0
2,"{анкерклин 6х60 мм starfix smp1837211, чохол t...",0.0
4,"{скалка деревянная фигурная сердечки 283см, см...",0.0
5,{светильник loft lsp9805},100.0
7,{32 жк монитор dell p3222qe 505267 },100.0


In [14]:
result = pd.merge(df, data.drop(columns='Name'), on='Number', how='inner')

In [15]:
result.head()

,Number,Barcode,Name,Similarity
0,1,3165140852395,далекомір bosch zamo ii металлическая упаковка...,0.0
1,1,4044996135779,отвертка sl5 х 100 мм crv двухкомпонентная рук...,0.0
2,1,4606008386861,пенал школьный 4645025 собачки малиновый феникс,0.0
3,2,4811208002736,анкерклин 6х60 мм starfix smp1837211,0.0
4,2,4811208002736,анкерклин 6х60 мм starfix smp1837211,0.0


In [16]:
result.to_csv('../data/result.csv')